<a href="https://colab.research.google.com/github/mrinal35/Email-Response-Assistant/blob/main/EmailResponese.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [81]:
!pip install spacy
!python -m spacy download en_core_web_sm
import nltk


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.8/12.8 MB 67.5 MB/s eta 0:00:00
✔ Download and installation successful
You can now load the package via spacy.load('en_core_web_sm')
⚠ Restart to reload dependencies
If you are in a Jupyter or Colab notebook, you may need to restart Python in
order to load all the package's dependencies. You can do this by selecting the
'Restart kernel' or 'Restart runtime' option.


In [82]:
import nltk
import spacy
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from nltk.sentiment.vader import SentimentIntensityAnalyzer
from sklearn.feature_extraction.text import TfidfVectorizer
import nltk
nltk.download('punkt')
nltk.download('stopwords')
nltk.download('wordnet')
nltk.download('vader_lexicon')
stop_words = set(stopwords.words('english'))
lemmatizer = WordNetLemmatizer()
sid = SentimentIntensityAnalyzer()
nlp = spacy.load('en_core_web_sm')


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package vader_lexicon to /root/nltk_data...
[nltk_data]   Package vader_lexicon is already up-to-date!


In [83]:
def tokenize_and_remove_stopwords(email_body):
    words = nltk.word_tokenize(email_body)
    return [word for word in words if word.lower() not in stop_words and word.isalnum()]

In [84]:
def pos_tagging(email_body):
    doc = nlp(email_body)
    return [(token.text, token.pos_) for token in doc]

In [85]:
def extract_relevant_entities(email_body):
    doc = nlp(email_body)
    relevant_entities = {
        'DATE': [],
        'TIME': [],
        'ORG': [],
        'PERSON': [],
        'MONEY': [],
        'EVENT': [],
        'GPE': [],
        'PRODUCT': [],
        'LAW': [],
        'CARDINAL': []
    }
    for ent in doc.ents:
        if ent.label_ in relevant_entities:
            relevant_entities[ent.label_].append(ent.text)
    return relevant_entities

In [86]:
def lemmatize_tokens(tokens):
    return [lemmatizer.lemmatize(word) for word in tokens]

In [87]:
def analyze_sentiment(email_body):
    sentiment_scores = sid.polarity_scores(email_body)
    compound_score = sentiment_scores['compound']
    if compound_score >= 0.05:
        return 'positive'
    elif compound_score <= -0.05:
        return 'negative'
    else:
        return 'neutral'


In [88]:
emails = [
    {
        'id': 1,
        'subject': 'Project Update Request',
        'body': '''
        Hey there,
        Just wanted to check in on how things are progressing with the project. I know we’ve had some setbacks recently, but I’m hoping you can provide a bit of an update before the day’s end. It’s not critical, but it would be really helpful to have this information soon. Also, can you highlight any areas where you're stuck or need assistance? Thanks in advance!
        ''',
        'sentiment': None,
        'urgency': None
    },
    {
        'id': 2,
        'subject': 'Scheduling a Meeting',
        'body': '''
        Hi,
        I was wondering if you had time next week to go over some of the points we discussed during our last call. It’s not urgent, but I believe getting this done sooner could help avoid some potential issues down the road. If you're busy, we can push it to later, but a quick sync-up would be ideal if possible.
        Best,
        John
        ''',
        'sentiment': None,
        'urgency': None
    },
    {
        'id': 3,
        'subject': 'Payment Reminder',
        'body': '''
        Hello,
        Just a gentle reminder regarding the outstanding payment that was due last month. I understand that things can sometimes fall through the cracks, but we would appreciate it if this could be resolved soon to avoid further delays. Let me know if there are any issues on your end, and we can discuss alternative arrangements if necessary.
        Regards,
        Sarah
        ''',
        'sentiment': None,
        'urgency': None
    },
    {
        'id': 4,
        'subject': 'Job Offer - Any Updates?',
        'body': '''
        Hi,
        I hope you’re doing well! I just wanted to follow up on the offer we sent out last week. We’re excited to move forward, but I wanted to know if you needed more time to consider, or if there’s anything else you’d like to discuss. There’s no rush, but it would be great to hear your thoughts whenever you’re ready.
        Warm regards,
        Emily
        ''',
        'sentiment': None,
        'urgency': None
    },
    {
        'id': 5,
        'subject': 'Critical Issue with the App',
        'body': '''
        Hi Team,
        I’ve been informed by one of our users that the application crashes whenever they try to access certain features. At this point, I’m not sure if this is an isolated incident or if more users are experiencing the same. It would be helpful if we could investigate this as soon as possible to determine the root cause before it escalates. Please prioritize this if it turns out to be widespread. Thanks.
        ''',
        'sentiment': None,
        'urgency': None
    }
]

In [89]:
for email in emails:
    email['tokens'] = tokenize_and_remove_stopwords(email['body'])
    email['pos_tags'] = pos_tagging(email['body'])
    email['entities'] = extract_relevant_entities(email['body'])
    email['lemmas'] = lemmatize_tokens(email['tokens'])
    email['sentiment'] = analyze_sentiment(email['body'])

email_bodies = [email['body'] for email in emails]
tfidf_matrix, feature_names = calculate_tfidf(email_bodies)

# Function to calculate urgency
def calculate_urgency(email, tfidf_matrix, feature_names, index):
    sentiment_scores = sid.polarity_scores(email['body'])
    sentiment_score = sentiment_scores['compound']
    sentiment_urgency = -sentiment_score

    tfidf_index = {word: i for i, word in enumerate(feature_names)}
    tfidf_score = 0
    if email['tokens']:
        tfidf_score = np.sum([tfidf_matrix[index, tfidf_index.get(word, -1)] for word in email['tokens'] if word in tfidf_index])

    entities = email['entities']
    entity_urgency = 0
    if entities['DATE'] or entities['TIME']:
        entity_urgency += 2
    if entities['ORG'] or entities['PERSON']:
        entity_urgency += 1
    if entities['MONEY'] or entities['PRODUCT'] or entities['LAW']:
        entity_urgency += 1.5
    if entities['EVENT']:
        entity_urgency += 1.2
    if entities['GPE'] or entities['LAW'] or entities['CARDINAL']:
        entity_urgency += 0.8

    urgency_score = sentiment_urgency + tfidf_score + entity_urgency
    return urgency_score

# Ranking emails based on urgency score
for idx, email in enumerate(emails):
    email['urgency_score'] = calculate_urgency(email, tfidf_matrix, feature_names, idx)

ranked_emails = sorted(emails, key=lambda x: x['urgency_score'], reverse=True)

for rank, email in enumerate(ranked_emails, 1):
    print(f"Rank {rank}: Email {email['id']} - Urgency Score: {email['urgency_score']}")
    print(f"  Subject: {email['subject']}")
    print(f"  Body: {email['body']}")
    print("-" )


Rank 1: Email 3 - Urgency Score: 6.486050359283643
  Subject: Payment Reminder
  Body: 
        Hello,
        Just a gentle reminder regarding the outstanding payment that was due last month. I understand that things can sometimes fall through the cracks, but we would appreciate it if this could be resolved soon to avoid further delays. Let me know if there are any issues on your end, and we can discuss alternative arrangements if necessary.
        Regards,
        Sarah
        
-
Rank 2: Email 1 - Urgency Score: 5.746331958826229
  Subject: Project Update Request
  Body: 
        Hey there, 
        Just wanted to check in on how things are progressing with the project. I know we’ve had some setbacks recently, but I’m hoping you can provide a bit of an update before the day’s end. It’s not critical, but it would be really helpful to have this information soon. Also, can you highlight any areas where you're stuck or need assistance? Thanks in advance!
        
-
Rank 3: Email 4 - Ur

In [90]:
from transformers import GPT2LMHeadModel, GPT2Tokenizer
model_name = 'gpt2'
tokenizer = GPT2Tokenizer.from_pretrained(model_name)
model = GPT2LMHeadModel.from_pretrained(model_name)

def generate_response(email_subject, email_body, max_length=150):

    context = f"Subject: {email_subject}\n\nBody: {email_body}\n\nResponse:"
    inputs = tokenizer.encode(context, return_tensors='pt')

    outputs = model.generate(inputs, max_length=max_length, num_return_sequences=1, no_repeat_ngram_size=2)
    response = tokenizer.decode(outputs[0], skip_special_tokens=True)

    response = response.split('Response:')[1].strip()
    return response


In [91]:
top_emails = ranked_emails[:1]
responses = {}

for email in top_emails:
    response = generate_response(email['subject'], email['body'])
    responses[email['id']] = response

for email_id, response in responses.items():
    print(f"Response for Email ID {email_id}:")
    print(response)
    print("\n---\n")

    modified_response = input(f"Edit response for Email ID {email_id} (leave blank to keep as is): ")
    if modified_response.strip():
        responses[email_id] = modified_response

print("Final Responses:")
for email_id, response in responses.items():
    print(f"Email ID {email_id}:")
    print(response)
    print("\n---\n")


The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


Response for Email ID 3:
...
 (1) I am sorry for the inconvenience. Please note that I have not received any payment for this item

---

Edit response for Email ID 3 (leave blank to keep as is): 
Final Responses:
Email ID 3:
...
 (1) I am sorry for the inconvenience. Please note that I have not received any payment for this item

---

